In [27]:
import tensorflow as tf
import numpy as np
(X_train, Y_train) , (X_test, Y_test) = tf.keras.datasets.mnist.load_data()

In [28]:
print(X_train.shape)
print(X_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [29]:
mean_vals = np.mean(X_train, axis = 0)
std_val = np.std(X_train)

In [30]:
X_train_centered = (X_train - mean_vals) / std_val
X_test_centered = (X_test - mean_vals) / std_val

In [31]:
X_train_centered = X_train_centered.reshape((X_train_centered.shape[0], -1)) 
X_test_centered = X_test_centered.reshape((X_test_centered.shape[0], -1)) 

print(X_train_centered.shape, Y_train.shape)
print(X_test_centered.shape, Y_test.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


In [6]:
np.random.seed(123)

훈련 데이터를 준비하기 위해 클레스 레이블  0 ~ 9 사이 정수를 원 핫 인코딩으로 변경한다.

In [7]:
Y_train_onehot = tf.keras.utils.to_categorical(Y_train)
print('처음 3개 레이블:', Y_train[:3])

처음 3개 레이블: [5 0 4]


In [8]:
print('처음 3개 레이블 원 핫\n:', Y_train_onehot[:3])

처음 3개 레이블 원 핫
: [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


### 피드 포워드 신경망 구성

이제 신경망을 구현한다. 간단하게 세 개의 완전 연결 층을 만든다.
처음 두개의 층은 하이퍼볼릭 탄젠트 활성 함수를 가진 50개의 은닉 유닛으로 이루어진다.
마지막 층은 열개의 클래스에 레이블에 해당하는 열개의 은닉 유닛을 가진다. 마지막 층은 각 클래스의 확률을 계산하기 위해 소프트맥스 함수를 사용한다.

In [9]:
model = tf.keras.models.Sequential()
model.add(
    tf.keras.layers.Dense(
        units = 50,
        input_dim = X_train_centered.shape[1],
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='tanh'))
model.add(
    tf.keras.layers.Dense(
        units = 50,
        input_dim = 50,
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='tanh'))
model.add(
tf.keras.layers.Dense(
units=Y_train_onehot.shape[1],
input_dim = 50,
kernel_initializer = 'glorot_uniform',
bias_initializer='zeros',
activation='softmax'))

먼저 Sequential 클래스를 사용하여 피드 포워드 신경망을 구현하는 새로운 모델을 초기화한다.
두개의 연속된 층에서 unit과 입력 유닛이 일치해야 한다. units, input_dim

Note. kernel_initializer = 'glorot_uniform'은 새로운 가중치 행렬 초기화 알고리즘 이다. 
글로럿세이비어 초기화는 심층 신경망을 안정적으로 초기화하는 방법이다. 절편은 일반적으로 0으로 초기화한다. 

지금까지 만든 모델 구조를 summary() 메소드를 사용하여 출력한다. 출력 시작 부분이 신경망 입력에 가까운 층이고 끝부분이 출력에 가까운 층이다.

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                39250     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                510       
Total params: 42,310
Trainable params: 42,310
Non-trainable params: 0
_________________________________________________________________


## 피드포워드 신경망 훈련

모델 구성을 마치면 훈련을 수행하기 전에 모델을 컴파일 해야 한다.
이 단계에서 최적화할 손실 함수를 정의하고 최적화에 사용할 경사 하강법 옵티마이저를 선택한다. 이전 장에서 사용해 보았던 확률적 경사 학아법 최적화를 선택한다. 에포크마다 학습률을 조절하기 위한 학습률 감쇠 상수와 모멘텀 값을 지정한다. 마지막으로 비용 함수를 catergorical_crossentropy로 설정한다.

이진 크로스 엔트로피는 로지스틱 손실함수의 기술적인 표현이다. 범주형 크로스 엔트로피는 소프트맥스를 사용하여 다중 클래스 예측으로 일반화한 것이다.

In [12]:
sgd_optimizer = tf.keras.optimizers.SGD(lr =0.001, decay = 1e-7, momentum = .9)
model.compile(optimizer = sgd_optimizer, loss='categorical_crossentropy')

모델을 컴파일 한 후 fit 메서드를 호출하여 훈련시킨다. 여기서 미니 배치 경사 하강법을 사용한다. 배치마다 담긴 훈련 샘플 개수는 64개다. 50번의 에포크 동안 MLP를 훈련시킨다. verbose를 1로 설정하여 훈련하는 동안 비용함수의 최적화 과정을 따라간다.

validation_split 매개변수는 유용하다. 0.1로 설정하면 훈련 데이터의 10퍼센트를 검정 데이터로 떼어낸다. 에포크마다 이 데이터로 검증 점수를 계산하므로 모델이 과대적합 되었는지 모니터링 할 수 있다.

In [14]:
history  = model.fit(X_train_centered, Y_train_onehot,
                    batch_size = 64, epochs = 50,
                    verbose = 1,
                    validation_split = 0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/50
54000/54000 [==============================] - 2s 42us/sample - loss: 0.7114 - val_loss: 0.3624
Epoch 2/50
54000/54000 [==============================] - 1s 27us/sample - loss: 0.3711 - val_loss: 0.2736
Epoch 3/50
54000/54000 [==============================] - 1s 27us/sample - loss: 0.3046 - val_loss: 0.2365
Epoch 4/50
54000/54000 [==============================] - 1s 26us/sample - loss: 0.2677 - val_loss: 0.2128
Epoch 5/50
54000/54000 [==============================] - 1s 27us/sample - loss: 0.2425 - val_loss: 0.1967
Epoch 6/50
54000/54000 [==============================] - 1s 26us/sample - loss: 0.2232 - val_loss: 0.1842
Epoch 7/50
54000/54000 [==============================] - 1s 26us/sample - loss: 0.2075 - val_loss: 0.1746
Epoch 8/50
54000/54000 [==============================] - 1s 27us/sample - loss: 0.1942 - val_loss: 0.1662
Epoch 9/50
54000/54000 [==============================] - 1s 27us/sample - loss: 0.1828 - val_l

훈련하는 동안 비용 함수 값을 출력하는 기능은 유용하다. 훈련 도중에 비용이 감소하는 지 여부를 빨리 확인해서 감소 하지 않는다면 하이퍼 파라미터를 튜닝하기 위해 알고리즘을 일찍 멈출 수 있다. 클래스 레이블을 예측하려면 predict_calsses 메서드를 사용하여 정수로 된 클래스 레이블을 얻을 수 있다.

In [33]:
Y_train_pred = model.predict_classes(X_train_centered, verbose =0)
correct_preds = np.sum(Y_train == Y_train_pred, axis = 0)
train_acc = correct_preds / Y_train.shape[0]

print('훈련 정확도: %.2f%%' % (train_acc * 100))

훈련 정확도: 98.96%


In [35]:
Y_test_pred = model.predict_classes(X_test_centered, verbose = 0)
correct_preds = np.sum(Y_test == Y_test_pred, axis = 0)
test_acc = correct_preds / Y_test.shape[0]

print('훈련 정확도: %.2f%%' % (train_acc * 100))

훈련 정확도: 98.96%


### 다층 신경망의 활성화 함수 선택

기술적으로 미분 가능하다면 어떤 함수라도 다층 신경망의 활성화 함수로 사용할 수 있다,
실제로는 은닉층이나 출력층에 선형 활성화 함수를 사용하는 것이 유용하지 않다. 
